The code used to originally pull the raw data from exchanges.

In [4]:
import ccxt
import pandas as pd
import statistics as stat
from math import pi
import time
import random
import os

#plotting
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import Axis
from bokeh.layouts import gridplot
from bokeh.plotting import figure

In [49]:
def create_ohlcv_df(data):
    header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    df = pd.DataFrame(data, columns=header)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'],unit='ms',origin='unix') # convert timestamp to datetime
    return df

In [61]:
# pulls "n_candles" number of candles with "c_size" candlesticks from a given start date
# for all symbol pairs on a given exchange and stores them in a pandas dataframe, 
# then saves them in csv's in the specified folder path
# returns a list with the symbols it could not fetch

# -- EXAMPLE INPUT --
# exchange = 'bittrex'
# from_date = '2018-04-20 00:00:00'
# n_candles = 480
# c_size = '1h'
# f_path = '../data'

def pull_data(exchange,from_date,n_candles,c_size,f_path,skip=False):
    count = 1
    msec = 1000
    hold = 30
    
    missing_symbols = []

    # -- create a folder --
    newpath = f_path+'/'+exchange+'/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    # -- load exchange --
    exc_instance = getattr(ccxt,exchange)()
    exc_instance.load_markets()
    from_timestamp = exc_instance.parse8601(from_date)
    
    # -- pull ohlcv --
    for symbol in exc_instance.symbols:
        for attempt in range(5): # 5 attempts max
            try:
                print('Pulling:',exchange,':',symbol, '[{}/{}]'.format(count,len(exc_instance.symbols)))
                data = exc_instance.fetch_ohlcv(symbol, c_size, from_timestamp, n_candles)
                
                # if < n_candles returned, skip this pair
                if len(data) < n_candles and skip is True:
                    continue
                
                # -- create DF --
                df = create_ohlcv_df(data)

                # -- save CSV --
                symbol = symbol.replace("/","-")
                filename = newpath+'{}_{}_[{}]-TO-[{}].csv'.format(exchange, symbol, df['Timestamp'].iloc[0], df['Timestamp'].iloc[-1])        
                df.to_csv(filename)
                
            except (ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout, IndexError) as error:
                print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
                time.sleep(hold)
            else: # if no error, proceed to next iteration
                break
        else: # we failed all attempts
            print('All attempts failed, skipping:',symbol)
            missing_symbols.append(symbol)
            continue
            
        count += 1   
        # -- wait for rate limit --
        time.sleep((exc_instance.rateLimit / msec) + 5) # rate limit +5 seconds to just to be safe

    # print out any symbols we could not obtain
    if len(missing_symbols) is not 0:
        print('Unable to obtain:',missing_symbols)
    
    
    return missing_symbols

In [58]:
# gets the symbols from a given "exchange", filtering out any symbol pairs provided in "without", and stores in a df
# additionally, if set to save and given a filepath, will save as .csv

# f_path = '../symbols'
def symbol_list_df(exchange, without=[], save=False, f_path=''):

    # -- load market --
    exc_instance = getattr(ccxt, exchange)()
    exc_instance.load_markets()
    
    # -- filter list --
    symbol_list = [s for s in exc_instance.symbols if s not in without]
    
    # -- save --
    if save is True:
      
        # -- create a folder --
        newpath = f_path+'/'
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        
        # -- create df --
        header = ['Symbols']
        df = pd.DataFrame(symbol_list, columns=header)
        filename = newpath+'{}_symbols.csv'.format(exchange)        
        df.to_csv(filename)
            
    return symbol_list

In [51]:
from_date = '2018-04-20 00:00:00'
exchanges = ['bittrex','binance','kraken','kucoin','lbank']

for e in exchanges:
    pull_data(e,from_date,480,'1h','../data')

Pulling: bittrex : 2GIVE/BTC [1/279]
Pulling: bittrex : ABY/BTC [2/279]
Pulling: bittrex : ADA/BTC [3/279]
Pulling: bittrex : ADA/ETH [4/279]
Pulling: bittrex : ADA/USDT [5/279]
Pulling: bittrex : ADT/BTC [6/279]
Pulling: bittrex : ADT/ETH [7/279]
Pulling: bittrex : ADX/BTC [8/279]
Pulling: bittrex : ADX/ETH [9/279]
Pulling: bittrex : AEON/BTC [10/279]
Pulling: bittrex : AMP/BTC [11/279]
Pulling: bittrex : ANT/BTC [12/279]
Pulling: bittrex : ANT/ETH [13/279]
Pulling: bittrex : ARDR/BTC [14/279]
Pulling: bittrex : ARK/BTC [15/279]
Pulling: bittrex : AUR/BTC [16/279]
Pulling: bittrex : BAT/BTC [17/279]
Pulling: bittrex : BAT/ETH [18/279]
Pulling: bittrex : BAY/BTC [19/279]
Pulling: bittrex : BCH/BTC [20/279]
Pulling: bittrex : BCH/ETH [21/279]
Pulling: bittrex : BCH/USDT [22/279]
Pulling: bittrex : BCPT/BTC [23/279]
Pulling: bittrex : BCPT/ETH [24/279]
Pulling: bittrex : BCY/BTC [25/279]
Pulling: bittrex : BITB/BTC [26/279]
Pulling: bittrex : BLITZ/BTC [27/279]
Pulling: bittrex : BLK/BTC

Pulling: bittrex : PAY/ETH [159/279]
Pulling: bittrex : PINK/BTC [160/279]
Pulling: bittrex : PIVX/BTC [161/279]
Got an error RequestTimeout ("bittrex GET https://bittrex.com/api/v2.0/pub/market/GetTicks?tickInterval=hour&marketName=BTC-PIVX HTTPSConnectionPool(host='bittrex.com', port=443): Read timed out. (read timeout=10)",) , retrying in 30 seconds...
Pulling: bittrex : PIVX/BTC [161/279]
Pulling: bittrex : POLY/BTC [162/279]
Pulling: bittrex : POLY/ETH [163/279]
Pulling: bittrex : POT/BTC [164/279]
Pulling: bittrex : POWR/BTC [165/279]
Pulling: bittrex : POWR/ETH [166/279]
Pulling: bittrex : PPC/BTC [167/279]
Pulling: bittrex : PRO/BTC [168/279]
Pulling: bittrex : PRO/ETH [169/279]
Pulling: bittrex : PTC/BTC [170/279]
Pulling: bittrex : PTOY/BTC [171/279]
Pulling: bittrex : PTOY/ETH [172/279]
Pulling: bittrex : QRL/BTC [173/279]
Pulling: bittrex : QRL/ETH [174/279]
Pulling: bittrex : QTUM/BTC [175/279]
Pulling: bittrex : QTUM/ETH [176/279]
Pulling: bittrex : QWARK/BTC [177/279]
Pu

Pulling: binance : BAT/BNB [26/315]
Pulling: binance : BAT/BTC [27/315]
Pulling: binance : BAT/ETH [28/315]
Pulling: binance : BCD/BTC [29/315]
Pulling: binance : BCD/ETH [30/315]
Pulling: binance : BCH/BNB [31/315]
Pulling: binance : BCH/BTC [32/315]
Pulling: binance : BCH/ETH [33/315]
Pulling: binance : BCH/USDT [34/315]
Pulling: binance : BCN/BNB [35/315]
Pulling: binance : BCN/BTC [36/315]
Pulling: binance : BCN/ETH [37/315]
Pulling: binance : BCPT/BNB [38/315]
Pulling: binance : BCPT/BTC [39/315]
Pulling: binance : BCPT/ETH [40/315]
Pulling: binance : BLZ/BNB [41/315]
Pulling: binance : BLZ/BTC [42/315]
Pulling: binance : BLZ/ETH [43/315]
Pulling: binance : BNB/BTC [44/315]
Pulling: binance : BNB/ETH [45/315]
Pulling: binance : BNB/USDT [46/315]
Pulling: binance : BNT/BTC [47/315]
Pulling: binance : BNT/ETH [48/315]
Pulling: binance : BQX/BTC [49/315]
Pulling: binance : BQX/ETH [50/315]
Pulling: binance : BRD/BNB [51/315]
Pulling: binance : BRD/BTC [52/315]
Pulling: binance : BRD/

Pulling: binance : SUB/ETH [248/315]
Pulling: binance : SYS/BNB [249/315]
Pulling: binance : SYS/BTC [250/315]
Pulling: binance : SYS/ETH [251/315]
Pulling: binance : TNB/BTC [252/315]
Pulling: binance : TNB/ETH [253/315]
Pulling: binance : TNT/BTC [254/315]
Pulling: binance : TNT/ETH [255/315]
Pulling: binance : TRIG/BNB [256/315]
Pulling: binance : TRIG/BTC [257/315]
Pulling: binance : TRIG/ETH [258/315]
Pulling: binance : TRX/BTC [259/315]
Pulling: binance : TRX/ETH [260/315]
Pulling: binance : VEN/BNB [261/315]
Pulling: binance : VEN/BTC [262/315]
Pulling: binance : VEN/ETH [263/315]
Pulling: binance : VIA/BNB [264/315]
Pulling: binance : VIA/BTC [265/315]
Pulling: binance : VIA/ETH [266/315]
Pulling: binance : VIB/BTC [267/315]
Pulling: binance : VIB/ETH [268/315]
Pulling: binance : VIBE/BTC [269/315]
Pulling: binance : VIBE/ETH [270/315]
Pulling: binance : WABI/BNB [271/315]
Pulling: binance : WABI/BTC [272/315]
Pulling: binance : WABI/ETH [273/315]
Pulling: binance : WAN/BNB [27

Got an error RequestTimeout ("kraken GET https://api.kraken.com/0/public/OHLC?pair=GNOXBT&interval=60&since=1524182400 HTTPSConnectionPool(host='api.kraken.com', port=443): Read timed out. (read timeout=10)",) , retrying in 30 seconds...
Pulling: kraken : GNO/BTC [26/68]
Pulling: kraken : GNO/ETH [27/68]
Pulling: kraken : GNO/EUR [28/68]
Pulling: kraken : GNO/USD [29/68]
Pulling: kraken : ICN/BTC [30/68]
Pulling: kraken : ICN/ETH [31/68]
Pulling: kraken : LTC/BTC [32/68]
Got an error RequestTimeout ("kraken GET https://api.kraken.com/0/public/OHLC?pair=XLTCXXBT&interval=60&since=1524182400 HTTPSConnectionPool(host='api.kraken.com', port=443): Read timed out. (read timeout=10)",) , retrying in 30 seconds...
Pulling: kraken : LTC/BTC [32/68]
Pulling: kraken : LTC/EUR [33/68]
Pulling: kraken : LTC/USD [34/68]
Pulling: kraken : MLN/BTC [35/68]
Pulling: kraken : MLN/ETH [36/68]
Pulling: kraken : REP/BTC [37/68]
Pulling: kraken : REP/ETH [38/68]
Pulling: kraken : REP/EUR [39/68]
Got an error

Pulling: kraken : XMR/BTC [46/68]
Got an error RequestTimeout ("kraken GET https://api.kraken.com/0/public/OHLC?pair=XXMRXXBT&interval=60&since=1524182400 HTTPSConnectionPool(host='api.kraken.com', port=443): Read timed out. (read timeout=10)",) , retrying in 30 seconds...
Pulling: kraken : XMR/BTC [46/68]
Pulling: kraken : XMR/EUR [47/68]
Pulling: kraken : XMR/USD [48/68]
Pulling: kraken : XRP/BTC [49/68]
Got an error RequestTimeout ("kraken GET https://api.kraken.com/0/public/OHLC?pair=XXRPXXBT&interval=60&since=1524182400 HTTPSConnectionPool(host='api.kraken.com', port=443): Read timed out. (read timeout=10)",) , retrying in 30 seconds...
Pulling: kraken : XRP/BTC [49/68]
Pulling: kraken : XRP/CAD [50/68]
Pulling: kraken : XRP/EUR [51/68]
Got an error RequestTimeout ("kraken GET https://api.kraken.com/0/public/OHLC?pair=XXRPZEUR&interval=60&since=1524182400 HTTPSConnectionPool(host='api.kraken.com', port=443): Read timed out. (read timeout=10)",) , retrying in 30 seconds...
Pulling:

Pulling: kucoin : ENJ/BTC [102/319]
Pulling: kucoin : ENJ/ETH [103/319]
Pulling: kucoin : EOS/BTC [104/319]
Pulling: kucoin : EOS/ETH [105/319]
Pulling: kucoin : EOS/NEO [106/319]
Pulling: kucoin : EOS/USDT [107/319]
Pulling: kucoin : ETC/BTC [108/319]
Pulling: kucoin : ETC/ETH [109/319]
Pulling: kucoin : ETC/KCS [110/319]
Pulling: kucoin : ETH/BTC [111/319]
Pulling: kucoin : ETH/USDT [112/319]
Pulling: kucoin : ETN/BTC [113/319]
Pulling: kucoin : ETN/ETH [114/319]
Pulling: kucoin : EVX/BTC [115/319]
Pulling: kucoin : EVX/ETH [116/319]
Pulling: kucoin : EXY/BTC [117/319]
Pulling: kucoin : EXY/ETH [118/319]
Pulling: kucoin : FLIXX/BTC [119/319]
Pulling: kucoin : FLIXX/ETH [120/319]
Pulling: kucoin : FOTA/BTC [121/319]
Pulling: kucoin : FOTA/ETH [122/319]
Pulling: kucoin : GAS/BTC [123/319]
Pulling: kucoin : GAS/KCS [124/319]
Pulling: kucoin : GAS/NEO [125/319]
Pulling: kucoin : GAT/BTC [126/319]
Pulling: kucoin : GAT/ETH [127/319]
Pulling: kucoin : GLA/BTC [128/319]
Pulling: kucoin : GL

Got an error IndexError ('single positional indexer is out-of-bounds',) , retrying in 30 seconds...
Pulling: kucoin : SRN-ETH [263/319]
Got an error ExchangeError ('kucoin No market symbol SRN-ETH',) , retrying in 30 seconds...
Pulling: kucoin : SRN-ETH [263/319]
Got an error ExchangeError ('kucoin No market symbol SRN-ETH',) , retrying in 30 seconds...
Pulling: kucoin : SRN-ETH [263/319]
Got an error ExchangeError ('kucoin No market symbol SRN-ETH',) , retrying in 30 seconds...
Pulling: kucoin : SRN-ETH [263/319]
Got an error ExchangeError ('kucoin No market symbol SRN-ETH',) , retrying in 30 seconds...
All attempts failed, skipping: SRN-ETH
Pulling: kucoin : STK/BTC [263/319]
Pulling: kucoin : STK/ETH [264/319]
Pulling: kucoin : STX/BTC [265/319]
Pulling: kucoin : STX/ETH [266/319]
Pulling: kucoin : SUB/BTC [267/319]
Pulling: kucoin : SUB/ETH [268/319]
Pulling: kucoin : TEL/BTC [269/319]
Pulling: kucoin : TEL/ETH [270/319]
Pulling: kucoin : TFL/BTC [271/319]
Pulling: kucoin : TFL/ETH

Got an error IndexError ('single positional indexer is out-of-bounds',) , retrying in 30 seconds...
Pulling: lbank : PTR-ETH [68/99]
Got an error ExchangeError ('lbank No market symbol PTR-ETH',) , retrying in 30 seconds...
Pulling: lbank : PTR-ETH [68/99]
Got an error ExchangeError ('lbank No market symbol PTR-ETH',) , retrying in 30 seconds...
Pulling: lbank : PTR-ETH [68/99]
Got an error ExchangeError ('lbank No market symbol PTR-ETH',) , retrying in 30 seconds...
Pulling: lbank : PTR-ETH [68/99]
Got an error ExchangeError ('lbank No market symbol PTR-ETH',) , retrying in 30 seconds...
All attempts failed, skipping: PTR-ETH
Pulling: lbank : PUT/ETH [68/99]
Pulling: lbank : PUT/QTUM [69/99]
Pulling: lbank : QBT/QTUM [70/99]
Pulling: lbank : QTUM/BTC [71/99]
Pulling: lbank : QTUM/ETH [72/99]
Pulling: lbank : QTUM/USDT [73/99]
Pulling: lbank : SBTC/BTC [74/99]
Pulling: lbank : SC/BTC [75/99]
Pulling: lbank : SEER/BTS [76/99]
Pulling: lbank : SEER/ETH [77/99]
Pulling: lbank : TGC/ETH [7

In [59]:
bittrex_w = ['XVC-BTC']
kraken_w = ['ETHCAD.d', 'ETHEUR.d', 'ETHGBP.d', 'ETHJPY.d', 'ETHUSD.d', 'ETHXBT.d', 'XBTCAD.d', 'XBTEUR.d', 'XBTGBP.d', 'XBTJPY.d', 'XBTUSD.d']
kucoin_w = ['BHC/BTC', 'BRD-BTC', 'BRD-ETH', 'MAN-BTC', 'MAN-ETH', 'SRN-BTC', 'SRN-ETH', 'TOMO-BTC', 'TOMO-ETH']
lbank_w = ['BCH/BTC', 'BCH/ETH', 'BCH/USDT', 'PTR-ETH', 'VET/BTC', 'VET/ETH', 'VET/USDT']

symbol_list_df('bittrex',bittrex_w,save=True,f_path='../symbols')
symbol_list_df('kraken',kraken_w,save=True,f_path='../symbols')
symbol_list_df('kucoin',kucoin_w,save=True,f_path='../symbols')
symbol_list_df('lbank',lbank_w,save=True,f_path='../symbols')

['AAC/ETH',
 'AAC/QTUM',
 'ABT/BTC',
 'ABT/ETH',
 'BCD/BTC',
 'BOT/QTUM',
 'BSB/ETH',
 'BTC/USDT',
 'BTF/BTC',
 'BTG/BTC',
 'BTM/ETH',
 'BTO/ETH',
 'BTS/BTC',
 'BTS/ETH',
 'CDC/ETH',
 'CHAT/QTUM',
 'CMT/BTC',
 'CMT/ETH',
 'CPX/BTC',
 'CPX/ETH',
 'CPX/NEO',
 'CS/ETH',
 'DASH/BTC',
 'DAX/BTC',
 'DAX/ETH',
 'DAX/USDT',
 'DBC/ETH',
 'DBC/NEO',
 'DCT/ETH',
 'DDD/ETH',
 'DDM/ETH',
 'EAI/ETH',
 'EKO/ETH',
 'ENT/QTUM',
 'EOS/ETH',
 'ETC/BTC',
 'ETH/BTC',
 'ETH/USDT',
 'EXT/ETH',
 'EXT/NEO',
 'FIL12/BTC',
 'FIL12/ETH',
 'FIL36/BTC',
 'FIL36/ETH',
 'FIL6/BTC',
 'FIL6/ETH',
 'GRAMS/ETH',
 'IHT/ETH',
 'IIC/ETH',
 'INC/ETH',
 'INK/ETH',
 'INK/QTUM',
 'LCH/ETH',
 'LTC/BTC',
 'MAN/ETH',
 'MDA/ETH',
 'MITH/ETH',
 'MITH/QTUM',
 'NAS/ETH',
 'NEO/BTC',
 'NEO/USDT',
 'OCN/ETH',
 'PLY/ETH',
 'PLY/QTUM',
 'PNT/BTC',
 'PNT/ETH',
 'PST/ETH',
 'PTR/ETH',
 'PUT/ETH',
 'PUT/QTUM',
 'QBT/QTUM',
 'QTUM/BTC',
 'QTUM/ETH',
 'QTUM/USDT',
 'SBTC/BTC',
 'SC/BTC',
 'SEER/BTS',
 'SEER/ETH',
 'TGC/ETH',
 'TKY/ETH',
 'TKY/